In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
datos_antigua_full = pd.read_csv("l_Unif.csv", delimiter=",", on_bad_lines='skip')
datos_nueva_full = pd.read_csv("a_Unif.csv", delimiter=",", on_bad_lines='skip')
datos_antigua = datos_antigua_full
datos_nueva = datos_nueva_full
N_datos = len(datos_nueva)
columnas_normalizar_a = list(datos_nueva.keys()[2:])
columnas_normalizar_l = list(datos_antigua.keys()[2:])
print(columnas_normalizar_a)

['bx', 'by', 'bz', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C11', 'C12', 'C13', 'C14', 'C15', 'C22', 'C23', 'C24', 'C25', 'C33', 'C34', 'C35', 'C44', 'C45', 'C55', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'eig_8', 'eig_9', 'eig_10', 'eig_11', 'eig_12', 'eig_13', 'eig_14', 'eig_15', 'eig_16', 'eig_17', 'eig_18', 'eig_19', 'eig_20', 'eig_21', 'eig_22', 'eig_23', 'eig_24', 'eig_25', 'eig_26', 'eig_27', 'eig_28', 'eig_29', 'eig_30', 'eig_31', 'eig_32', 'eig_33', 'eig_34', 'eig_35', 'eig_36', 'eig_37', 'eig_38', 'eig_39', 'eig_40', 'eig_41', 'eig_42', 'eig_43', 'eig_44', 'eig_45', 'eig_46', 'eig_47', 'eig_48', 'eig_49', 'eig_50', 'eig_51', 'eig_52', 'eig_53', 'eig_54', 'eig_55', 'eig_56', 'eig_57', 'eig_58', 'eig_59', 'eig_60', 'eig_61', 'eig_62', 'eig_63', 'eig_64', 'eig_65', 'eig_66', 'eig_67', 'eig_68', 'eig_69', 'eig_70', 'eig_71', 'eig_72', 'eig_73', 'eig_74', 'eig_75', 'eig_76', 'eig_77', 'eig_78', 'eig_79', 'eig_80', 'eig_81', 'eig_82', 'eig_83', 'eig

In [3]:
def one_hottear(d_frame, cols_discretas):
    cols_nuevas = []
    for column in cols_discretas:
        posibles_valores = set(d_frame[column])
        for i in posibles_valores:
            d_frame[column + str(i)] = 0
            d_frame.loc[d_frame[column] == i, column + str(i)] = 1
            cols_nuevas.append(column + str(i))
        #fin for 
        del d_frame[column]
    #fin for 
    d_frame.loc[:, cols_nuevas + [col for col in d_frame.columns if col not in cols_nuevas]]
#fin función

In [4]:
one_hottear(datos_antigua, ["# Shape", "Cry_st"])
one_hottear(datos_nueva, ["# Shape", "Cry_st"])

In [5]:
Datos_train_l, Datos_test_l = train_test_split(datos_antigua, test_size = 0.4)
Datos_train_a, Datos_test_a = train_test_split(datos_nueva, test_size = 0.4)

In [6]:
def normalizar(dataSet, features, parametros = []):
    """
    Esta función resta la media y luego divide entre la desviación estandar de cada dato en cada columna. Si se le especifica la media y la desviación en los parámetros 
    Usa esa media y desviación dada, de lo contrario, calcula la media y la desviación de cada columna especificada en "features"
    @input: dataSet <pd.DataFrame>: Tabla de datos a la que se le va a dar normalización a la columans dadas
    @input: features <list>: lista de "features" o columnas que se van a normalizar
    @input: (opcional) parametros <dict>: En caso de que se quiera normalizar con una media y una desviación dada se especifican en un diccionario así: {'mileage':{'media': 2.9, 'desviacion': 7}}
    """
    if len(parametros) == 0:
        for feature in features:
            media = np.mean(dataSet[feature])
            desviacion = np.std(dataSet[feature])
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
    #fin if 
    else:
        for feature in features:
            media = parametros[feature]['media']
            desviacion = parametros[feature]['desviacion']
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
#fin función

In [7]:
Estadisticos_train_l= dict(map(lambda x: (x, {'media': np.mean(Datos_train_l[x]), 'desviacion': np.std(Datos_train_l[x])}), Datos_train_l.keys()))
Estadisticos_train_a= dict(map(lambda x: (x, {'media': np.mean(Datos_train_a[x]), 'desviacion': np.std(Datos_train_a[x])}), Datos_train_a.keys()))

In [8]:
normalizar(Datos_train_l, columnas_normalizar_l)
normalizar(Datos_train_a, columnas_normalizar_a)
normalizar(Datos_test_l, columnas_normalizar_l, Estadisticos_train_l)
normalizar(Datos_test_a, columnas_normalizar_a, Estadisticos_train_a)

In [9]:
"""
TODO: 
1) Hacer una lista con los features que serán nuestros X. Escoger cualquiera de los CXX y ese será nuestro 'y' (luego haremos un loop para cubrir todos los CXX)
2) Hacer una regresión lineal con cada uno de los CXX y reportar un MSE por cada uno de los CXX
NO HAY QUE HACER VALIDACIÓN!!
"""

,Density,Lx,Ly,Lz,C00,C01,C02,C03,C04,C05,...,(omega^2)_497,(omega^2)_498,(omega^2)_499,# Shape0.0,# Shape1.0,# Shape2.0,Cry_st0.0,Cry_st1.0,Cry_st2.0,Cry_st3.0
7693,1.698301,1.472468,-0.283582,0.602766,0.039642,-0.409427,-0.321255,0.0,0.0,0.0,...,-0.343130,-0.343190,-0.343312,1,0,0,1,0,0,0
16739,-1.615754,-0.097388,0.325519,0.940418,1.063241,1.094931,1.094923,0.0,0.0,0.0,...,-0.100223,-0.100728,-0.100941,0,0,1,0,0,0,1
8188,1.122501,-0.304500,-0.124669,-0.872415,-0.846055,-0.013781,-0.152736,0.0,0.0,0.0,...,-0.317146,-0.317280,-0.317390,1,0,0,0,1,0,0
6047,-1.677829,1.172536,-0.472849,1.334082,1.532511,0.641276,0.641355,0.0,0.0,0.0,...,-0.152664,-0.152923,-0.152812,1,0,0,0,0,1,0
26184,1.509066,-0.821901,1.340533,1.110818,0.503086,3.167070,3.166661,0.0,0.0,0.0,...,-0.345295,-0.345355,-0.345477,1,0,0,0,0,0,1
